In [1]:

from typing import Sequence

import torch
import numpy as np
from models import MLPModel, FancyMLPModel
from agents import Agent
from collectors import Memory, CrowdCollector
from environments import UnityCrowdEnv, UnitySimpleCrowdEnv
from policy_optimization import CrowdPPOptimizer
from trainers import PPOCrowdTrainer

from tqdm import tqdm, trange
from mlagents_envs.environment import UnityEnvironment

from utils import transpose_batch, concat_batches, concat_crowd_batch, tanh_norm, atanh_unnorm

from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel

import seaborn as sns
import matplotlib.pyplot as plt

from utils import discount_rewards_to_go, get_episode_lens

In [2]:

env = UnitySimpleCrowdEnv(file_name=None)




env.engine_channel.set_configuration_parameters(time_scale=100)

This means that some features may not work unless you upgrade the package with the lower version.Please find the versions that work best together from our release page.
https://github.com/Unity-Technologies/ml-agents/releases


In [3]:
action_range = (
    torch.tensor([-.3, -1.]),
    torch.tensor([ 1.,  1.])
)

In [4]:
model = MLPModel({
    "input_size": 94,
})

agent = Agent(model, action_range=action_range)

In [5]:
collector = CrowdCollector(agent, env)

In [6]:
batch = collector.collect_data(500, disable_tqdm=False)

  0%|          | 0/500 [00:00<?, ?it/s]/Users/redtachyon/projects/CrowdAI/training/utils.py:432: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  values = np.array([value_dict[key] for key in keys])
  0%|          | 0/500 [00:00<?, ?it/s]


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [7]:
import pdb; pdb.pm()

> /Users/redtachyon/projects/CrowdAI/training/collectors.py(200)collect_data()
    198 
    199             obs_array, agent_keys = pack(obs_dict)
--> 200             obs_tensor = torch.tensor(obs_array)
    201 
    202             # Centralize the action computation for better parallelization

array([array([ 0.        ,  0.        ,  1.        ,  1.        ,  0.        ,
        0.        ,  1.        ,  1.        ,  0.        ,  0.        ,
        1.        ,  1.        ,  0.        ,  0.        ,  1.        ,
        1.        ,  0.        ,  0.        ,  1.        ,  1.        ,
        0.        ,  0.        ,  1.        ,  1.        ,  0.        ,
        0.        ,  1.        ,  1.        ,  0.        ,  0.        ,
        1.        ,  1.        ,  0.        ,  0.        ,  1.        ,
        1.        ,  0.        ,  0.        ,  1.        ,  1.        ,
        0.        ,  0.        ,  1.        ,  1.        ,  0.        ,
        0.        ,  1.        ,  1.        ,  0

In [9]:
env.close()